### 프림 알고리즘  

#### 조건  
>무방향 그래프  
component는 하나(모든 정점이 연결된 상태)  

#### 알고리즘  
1. 주어진 그래프에 있으면서 새로 만들어질 MST에 없는 정점 선택  
2. 선택된 정점과 연결된 정점들의 가중치는 업데이트하고 엣지는 우선순위 큐에 삽입  
3. 우선순위 큐에서 가중치가 가장 작은 엣지를 선택. 선택된 정점은 MST에 추가  
4. 모든 정점이 MST에 추가될때까지 1,2,3 반복 끝.  


In [2]:
class Element:
    def __init__(self, v, w, _from):
        # 가중치를 키로 사용한다
        self.w=w
        self.v=v
        self._from=_from

class MinHeap:
    MAX_ELEMENTS=200
    def __init__(self):
        self.arr=[None for i in range(self.MAX_ELEMENTS)]
        self.heapsize=0
        #정점이 arr에 위치한 현재 인덱스
        self.pos=[None for i in range(self.MAX_ELEMENTS)]

    def is_empty(self):
        if self.heapsize==0:
            return True
        return False

    def is_full(self):
        if self.heapsize>=self.MAX_ELEMENTS:
            return True
        return False

    def parent(self, idx):
        return idx >> 1

    def left(self, idx):
        return idx << 1

    def right(self, idx):
        return (idx << 1) + 1

    def push(self, item):
        if self.is_full():
            raise IndexError("the heap is full!!")

        self.heapsize+=1
        cur_idx=self.heapsize

        while cur_idx!=1 and item.w < self.arr[self.parent(cur_idx)].w: 
            self.arr[cur_idx]=self.arr[self.parent(cur_idx)]
            # pos의 인덱스는 정점, arr는 weight를 키로 만든 최소 힙
            self.pos[self.arr[cur_idx].v]=cur_idx

            cur_idx=self.parent(cur_idx)

        self.arr[cur_idx]=item
        self.pos[item.v]=cur_idx

    def pop(self):
        if self.is_empty():
            return None

        rem_elem=self.arr[1]

        temp=self.arr[self.heapsize]
        self.heapsize-=1

        cur_idx=1
        child=self.left(cur_idx)

        while child <= self.heapsize:
            if child < self.heapsize and \
                self.arr[self.left(cur_idx)].w > self.arr[self.right(cur_idx)].w:
                child=self.right(cur_idx)
            
            if temp.w <= self.arr[child].w:
                break

            self.arr[cur_idx]=self.arr[child]
            self.pos[self.arr[cur_idx].v]=cur_idx

            cur_idx=child
            child=self.left(cur_idx)
        
        self.arr[cur_idx]=temp
        self.pos[temp.v]=cur_idx

        return rem_elem

    def decrease_weight(self, new_elem):
        cur=self.pos[new_elem.v]

        while cur!= 1 and new_elem.w < self.arr[self.parent(cur)].w:
            self.arr[cur]=self.arr[self.parent(cur)]
            self.pos[self.arr[cur].v]=cur    

            cur=self.parent(cur)

        self.arr[cur]=new_elem
        self.pos[new_elem.v]=cur


In [3]:
import math

class Edge:
    def __init__(self, u, v, w):
        self.u=u
        self.v=v
        self.w=w

class Graph:
    def __init__(self, vertex_num):
        self.adj_list=[[] for _ in range(vertex_num)]
        self.edge_list=[]

        self.vertex_num=vertex_num

    def add_edge(self, u, v, w):
        # (정점, 에지의 가중치)를 인접 리스트에 추가
        self.adj_list[u].append((v, w))
        self.adj_list[v].append((u, w))

        self.edge_list.append(Edge(u, v, w))

    def MST_prim(self):
        mst=Graph(self.vertex_num)

        w_list=[math.inf for _ in range(self.vertex_num)]

        TV=set()
        h=MinHeap()

        for i in range(1, self.vertex_num):
            h.push(Element(i, math.inf, None))

        w_list[0]=0
        h.push(Element(0, 0, None))

        while not h.is_empty():
            elem_v=h.pop()
            v=elem_v.v 
            w=elem_v.w 
            _from=elem_v._from
            
            TV.add(v)
            if _from != None:
                mst.add_edge(v, _from, w)
            
            adj_v=self.adj_list[v]
            for u, w_u_v in adj_v:
                if u not in TV and w_u_v < w_list[u]:
                    w_list[u]=w_u_v
                    h.decrease_weight(Element(u, w_u_v, v)) 

        return mst

    def print_edges(self):
        for edge in self.edge_list:
            print("({}, {}) : {}".format(edge.u, edge.v, edge.w))

if __name__=="__main__":
    g=Graph(6)

    g.add_edge(0, 1, 10)
    g.add_edge(0, 2, 2)
    g.add_edge(0, 3, 8)
    g.add_edge(1, 2, 5)
    g.add_edge(1, 4, 12)
    g.add_edge(2, 3, 7)
    g.add_edge(2, 4, 17)
    g.add_edge(3, 4, 4)
    g.add_edge(3, 5, 14)

    mst=g.MST_prim()

    mst.print_edges()
    

(2, 0) : 2
(1, 2) : 5
(3, 2) : 7
(4, 3) : 4
(5, 3) : 14
